# Quick start

## Installation

The Python module can be directly installed from [PyPI](https://pypi.org/project/anta-database/) with:

    pip install anta_database


This Python module is designed to query and visualized data in a AntADatabase folder. The latter has to be first compiled using published IRH datasets, or you can contact for downloading it.
You can already have a look at this guide to have an idea of the features of this tool. This Jupyter Notebook can be directly downloaded (top bar) and ran locally (assuming you had downloaded or compiled the AntADatabase folder).


## Browsing the database

First, initialize the Database class by providing the full path to the AntADatabase:

In [ ]:
from anta_database import Database

db = Database('/home/anthe/documents/data/isochrones/AntADatabase/')

Use the query() function to browse the database. 'query()' without argument will return all metadata contained in the database:

In [ ]:
db.query()

The query() function takes as argument:
- author: author(s) of the datasets of interest
- age: age(s) in yrs before present of the layer(s) of interest
- var: variables(s) other than age depth such as 'IceThk', 'SurfElev', 'BedElev' or 'BasalUnit'
- trace_id: ID of a particular trace. This is useful for explicit trace IDs such as 'DC_LDC_DIVIDE'

One can also combine field queries as well as providing lists. Use '%' for searching by regular expressions. Here are a few examples:

In [ ]:
#Examples of queries:
db.query(author='Cavitte_2020') # all data from Cavitte et al. 2020
db.query(age='38100') # all datasets with the 38.1ka isochrone
db.query(var='IceThk') # all datasets with IceThk variable
db.query(trace_id='DC_LDC_DIVIDE') # all layers with the trace ID DC_LDC_DIVIDE
db.query(trace_id='%WSB%') # all traces with WSB in the trace ID
db.query(author=['Franke_2025', 'Winter_2018'], age='38100') # example of multiple criteria

## Visualization

Use the results of the query in the plotting functions:

In [ ]:
results = db.query(age=['38100','38000'])
db.plot.author(results,
                downscale_factor=100, # downscale the datasets n times, which makes no visual difference but it is much lighter
                title='AntArchitecture 38(.1)ka layer',
                xlim=(-500, 2400), ylim=(-2200, 2200), # set the plot extent in km
                scale_factor=0.7, # adjust the size of the plot
                latex=True, # use latex compilers for plotting if you have them installed on your system
                # save='AntA_38ka.pdf', # Uncomment to save the figure, otherwise it we visualize with pyplot
                )

Current implemented plotting functions are:
- plot.author(): plot locations of the data, with different colors for the different datasets (example above)
- plot.depth(): color-coded IRH depth scatter plot. One should provide a valid layer age.
- plot.var(): color-coded scatter plot for the additional variables. One should use it for plotting 'IceThk', 'SurfElev', 'BedElev', 'BasalUnit' or 'IRHDensity'
- plot.trace_id(): color-coded trace IDs. Useful for identifying specific traces of interest.

In Jupyter Notebook, use '%matplotlib qt' or '%matplotlib widget' depending on your IDE, to switch to the matplotlib widget that allows you to zoom in etc.
Use '%matplotlib inline' (default) to plot the figure in the notebook

In [ ]:
# %matplotlib widget
%matplotlib inline
results = db.query(age=['38100','38000'])
db.plot.depth(results, title='AntArchitecture 38(.1)ka isochrone depth',
                downscale_factor=100,
                xlim=(-500, 2400),
                ylim=(-2200, 2200),
                scale_factor=0.7,
                latex=True,
                # save='AntA_38ka_depth.png'
                )

In [ ]:
results = db.query(var='IRHDensity')
db.plot.var(results, title='IRH Density over the Antarctic Ice Sheet',
                downscale_factor=100,
                scale_factor=1,
                latex=True,
                )

In [ ]:
results = db.query(author='Winter%', var='IRHDensity', trace_id=['EPICA%'])
db.plot.trace_id(results, title='Winter et al. 2018 - EPICA',
                xlim=(-500, 1000),
                ylim=(1000, 2200),
                latex=True,
                )

For an overview of the database, I suggest use plot.author() providing var='IRHDensity'. Indeed this variable is computed for each trace, giving the spatial span of the data: 

In [ ]:
results = db.query(var='IRHDensity')
db.plot.author(results, title='Overview of the AntADatabase',
                downscale_factor=100,
                latex=True,
                # save='all_data.png'
                )


Other possible arguments for the plot methods:
- cmap: provide your colormap of choice (as LinearSegmentedColormap). Tip: 'import colormaps as cmaps' for a large choice of colormaps (see [Colormaps docs](https://pratiman-91.github.io/colormaps/)) 
- marker_size: for setting larger or smaller markers on the scatter plots

## Generate data from the database

Note: This part could be developed further in the future if there is the need. But for now, I am designing a separate Python module for constraining my ice sheet model of use, which is tailored to this database and other parallel processing libraries.

The data_generator() function reads the query and 'yield' the dataframes for later use.
Here is a quick example of how this can be used for computing the mean layer depth:

In [ ]:
results = db.query(age=['38100'])
lazy_dfs = db.data_generator()

import numpy as np
mean_depth_trs = []
min_depth = float('inf')
max_depth = float('-inf')
for df, md in lazy_dfs:
    depth_values = df['IRHDepth']
    mean_depth_trs.append(np.mean(depth_values))
    min_depth = min(min_depth, min(depth_values))
    max_depth = max(max_depth, max(depth_values))


mean_depth = np.mean(mean_depth_trs)
std_dev = np.std(mean_depth_trs, ddof=1)
print(f"The mean depth of the 38ka isochrone across East Antarctica is {round(mean_depth, 2)} m ranging from {round(min_depth, 2)} m to {round(max_depth, 2)} m.")

In the for loop, 'df' is an individual dataframe from the database, corresponding to a single layer of a single trace from a single dataset. The 'md' stores the unique metadata from the current 'df'. One can then associate the metadata with the current df, for example:

In [ ]:
results = db.query(author='Cavitte_2020', trace_id='DC_LDC_DIVIDE')
lazy_dfs = db.data_generator(results)

import matplotlib.pyplot as plt
# from matplotlib import rc
# rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
# rc('text', usetex=True) # Do you use latex?

for df, md in lazy_dfs:
    if md['age'] is not None:
        plt.plot(df.distance/1000, df['IRHDepth'], label=md['age'])
    if md['var'] == 'IceThk':
        plt.plot(df.distance/1000, df['IceThk'], linewidth=2, color='k', label='Bedrock')

plt.legend(ncol=2)
plt.xlabel('Distance (km)')
plt.ylabel('Depth (m)')
plt.ylim(4000, 0)
plt.title('Isochronal structure along DC_LDC_DIVIDE transect')
plt.show()

### Downscale the data

The downscale_factor argument in the data_generator reduces the size of the generated data by n times. This is useful when dealing with large data where the downscaling has a neglectable influence on the results.
Another solution, probably more straightforward, is the downsample_distance option, which allows to provide a distance in meters by which the data should be average along the transects:

In [ ]:
import matplotlib.pyplot as plt

results = db.query(author='Cavitte_2020', age='278000', trace_id='DC_LDC_DIVIDE')
original = db.data_generator(results)
downscale_10 = db.data_generator(results, downscale_factor=100)
downsampled_by_distance = db.data_generator(results, downsample_distance=5000)

list = [original, downscale_10, downsampled_by_distance]
labels = ['original', 'downscale 100', 'downsample 5km']

for i, lazy_dfs in enumerate(list):
    for df, md in lazy_dfs:
        plt.plot(df.distance, df['IRHDepth'],label=labels[i], linestyle='--')

plt.ylim(3000, 0)
plt.xlabel('Distance along DC_LDC_DIVIDE (km)')
plt.ylabel('Depth (m)')
plt.legend()
plt.title('Different downscaling methods for Cavitte 2020 278ka along DC_LDC_DIVIDE')
plt.show()

# Pro tips

The Database methods always keep the last query in memory. This means that one does not have to actually pass argument.
Example:

In [ ]:
db.query(var='IRHDensity')
# db.plot.var() # This is equivalent to the overview plot above. Note that without downscaling it is very heavy to plot.

import numpy as np
mean_density_trs = []
min_density = float('inf')
max_density = float('-inf')
for df, md in db.data_generator():    # without explicitly passing results, it uses the last query.
    density_values = df['IRHDensity']
    mean_density_trs.append(np.mean(density_values))
    min_density = min(min_density, min(density_values))
    max_density = max(max_density, max(density_values))

mean_density = np.mean(mean_density_trs)
print(f"The average number of picked layers per data point in the AntADatabase is {int(mean_density)}, ranging from {int(min_density)} to {int(max_density)}. ")
